In [1]:
import os
import sys
from random import sample
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
from dotenv import load_dotenv
import pandas as pd
import datetime
from dateutil import parser as date_parser
import json
from pprint import pprint
from functions_2 import *

In [2]:
# Environment vars
load_dotenv()
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [3]:
# Hardcoded date
today = datetime.datetime(year=2022, month=4, day=7)
today_str = datetime_string(today, "/")

In [4]:
# Instantiate blob service client
try:
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
except Exception as e:
    print(f'Unable to connect to BlobServiceClient: {e}')

In [5]:
# Fetching users, accounts, and chargebee subscriptions.
# Merging all tables into one.
df = get_users_accounts_subscriptions_data(blob_service_client, today_str)

Fetching user data...
<class 'pandas.core.frame.DataFrame'>
Int64Index: 88329 entries, 3 to 90338
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   _id.$oid                           88329 non-null  object 
 1   accountId.$oid                     88329 non-null  object 
 2   language                           88289 non-null  float64
 3   dateCreated.$date                  88329 non-null  int64  
 4   email                              88329 non-null  object 
 5   auth0Cache.usermetadata.lastName   88248 non-null  object 
 6   auth0Cache.usermetadata.firstName  88256 non-null  object 
 7   subscriptionType                   88329 non-null  object 
 8   isPrimary                          88329 non-null  bool   
 9   auth0Cache.usermetadata.demo       161 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 6.8+ MB
None
Fetching account data...


In [6]:
df["user|dateCreated.$date"].head()
df[df["user|dateCreated.$date"] != None].head()

,account|_id.$oid,account|name,account|domain,account|isActive,account|type,account|trialEnds.$date,account|dateCreated.$date,account|ownerId.$oid,account|edition,account|subscriptionId,...,user|language,user|dateCreated.$date,user|email,user|auth0Cache.usermetadata.lastName,user|auth0Cache.usermetadata.firstName,user|subscriptionType,user|isPrimary,user|auth0Cache.usermetadata.demo,account|id,account|users_count
0,5b14ee04854ff50007e2a077,Azavista,azavista,True,1.0,1528897421000,2018-06-04 07:45:08.977,5b14ee05854ff50007e2a078,gtmhub-summit,1mk51ZXQtzEmDUIfF,...,0.0,2018-06-07 14:13:32.294000,jp@azavista.com,van der Kuijl,JP,regular,True,NaN,5b14ee04854ff50007e2a077,2
1,5b14ee04854ff50007e2a077,Azavista,azavista,True,1.0,1528897421000,2018-06-04 07:45:08.977,5b14ee05854ff50007e2a078,gtmhub-summit,1mk51ZXQtzEmDUIfF,...,0.0,2018-06-04 07:45:09,gabriel.thomaidis@azavista.com,Thomaidis,Gabriel,regular,True,NaN,5b14ee04854ff50007e2a077,2
2,5b1e3680d8c5b500075ecfd3,wetransform GmbH,wetransform,True,1.0,1529311488473,2018-06-11 08:44:48.473,5b1e3680d8c5b500075ecfd4,gtmhub-summit,1mkVvueQuePTWUOmB,...,0.0,2018-06-11 08:44:48.498000,tr@wetransform.to,Reitz,Thorsten,regular,True,NaN,5b1e3680d8c5b500075ecfd3,4
3,5b1e3680d8c5b500075ecfd3,wetransform GmbH,wetransform,True,1.0,1529311488473,2018-06-11 08:44:48.473,5b1e3680d8c5b500075ecfd4,gtmhub-summit,1mkVvueQuePTWUOmB,...,0.0,2018-06-12 12:20:29.635000,st@wetransform.to,Templer,Simon,regular,True,NaN,5b1e3680d8c5b500075ecfd3,4
4,5b1e3680d8c5b500075ecfd3,wetransform GmbH,wetransform,True,1.0,1529311488473,2018-06-11 08:44:48.473,5b1e3680d8c5b500075ecfd4,gtmhub-summit,1mkVvueQuePTWUOmB,...,0.0,2019-06-25 15:40:06.882000,ch@wetransform.to,Hönn,Christopher,regular,True,NaN,5b1e3680d8c5b500075ecfd3,4


In [7]:
# Last 60 days
threshold_date = today - datetime.timedelta(days=60)

In [8]:
# Filter on users created at least 60 days ago
df = df[df["user|dateCreated.$date"] <= threshold_date]

In [9]:
def merge_col_values(row):
    print(row)

    new_value = row

    # row["user|auth0Cache.usermetadata.firstName"] + "|" + row["user|auth0Cache.usermetadata.lastName"]

    return new_value

In [10]:
# df["temp"] = df.apply(
#     lambda row: merge_col_values
# )

In [11]:
# Number of unique users existing for at least 60 days
df.shape[0]

35074

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35074 entries, 0 to 42634
Data columns (total 53 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   account|_id.$oid                        35074 non-null  object        
 1   account|name                            35074 non-null  object        
 2   account|domain                          35074 non-null  object        
 3   account|isActive                        35074 non-null  bool          
 4   account|type                            35074 non-null  float64       
 5   account|trialEnds.$date                 35074 non-null  int64         
 6   account|dateCreated.$date               35074 non-null  datetime64[ns]
 7   account|ownerId.$oid                    35074 non-null  object        
 8   account|edition                         35074 non-null  object        
 9   account|subscriptionId                  35074 non-

In [13]:

# existing_data = pd.read_csv("sample_users_180+days_data_2022-04-07.csv")

# existing_data["temp"] = existing_data.apply(
#     lambda row: row["user.first_name"] + "|" + row["user.last_name"]
# )

# existing_data.info()

In [14]:
# Remove the rows belonging to the users who were previously selected.
# df = df[
#     ~df["temp"].isin(existing_data["temp"])
# ]

# df.shape

In [15]:
# Sample 400 users, but no more than 1 user per account

# Sample 400 unique accounts
# sample_accounts = sample(df["account|_id.$oid"].unique().tolist(), 400)

# Choose 1 user for each of the accounts sampled
# sample_users = [
#     sample(
#         df[df["account|_id.$oid"] == acc]["user|_id.$oid"].tolist(), 
#         1
#     )[0]
#     for acc in sample_accounts
# ]

# len(sample_users)

In [16]:
# sample_users[:10]

In [17]:
# df = df[df["user|_id.$oid"].isin(sample_users)]

In [18]:
df.head()

,account|_id.$oid,account|name,account|domain,account|isActive,account|type,account|trialEnds.$date,account|dateCreated.$date,account|ownerId.$oid,account|edition,account|subscriptionId,...,user|language,user|dateCreated.$date,user|email,user|auth0Cache.usermetadata.lastName,user|auth0Cache.usermetadata.firstName,user|subscriptionType,user|isPrimary,user|auth0Cache.usermetadata.demo,account|id,account|users_count
0,5b14ee04854ff50007e2a077,Azavista,azavista,True,1.0,1528897421000,2018-06-04 07:45:08.977,5b14ee05854ff50007e2a078,gtmhub-summit,1mk51ZXQtzEmDUIfF,...,0.0,2018-06-07 14:13:32.294000,jp@azavista.com,van der Kuijl,JP,regular,True,NaN,5b14ee04854ff50007e2a077,2
1,5b14ee04854ff50007e2a077,Azavista,azavista,True,1.0,1528897421000,2018-06-04 07:45:08.977,5b14ee05854ff50007e2a078,gtmhub-summit,1mk51ZXQtzEmDUIfF,...,0.0,2018-06-04 07:45:09,gabriel.thomaidis@azavista.com,Thomaidis,Gabriel,regular,True,NaN,5b14ee04854ff50007e2a077,2
2,5b1e3680d8c5b500075ecfd3,wetransform GmbH,wetransform,True,1.0,1529311488473,2018-06-11 08:44:48.473,5b1e3680d8c5b500075ecfd4,gtmhub-summit,1mkVvueQuePTWUOmB,...,0.0,2018-06-11 08:44:48.498000,tr@wetransform.to,Reitz,Thorsten,regular,True,NaN,5b1e3680d8c5b500075ecfd3,4
3,5b1e3680d8c5b500075ecfd3,wetransform GmbH,wetransform,True,1.0,1529311488473,2018-06-11 08:44:48.473,5b1e3680d8c5b500075ecfd4,gtmhub-summit,1mkVvueQuePTWUOmB,...,0.0,2018-06-12 12:20:29.635000,st@wetransform.to,Templer,Simon,regular,True,NaN,5b1e3680d8c5b500075ecfd3,4
4,5b1e3680d8c5b500075ecfd3,wetransform GmbH,wetransform,True,1.0,1529311488473,2018-06-11 08:44:48.473,5b1e3680d8c5b500075ecfd4,gtmhub-summit,1mkVvueQuePTWUOmB,...,0.0,2019-06-25 15:40:06.882000,ch@wetransform.to,Hönn,Christopher,regular,True,NaN,5b1e3680d8c5b500075ecfd3,4


In [19]:
df.shape

(35074, 53)

In [20]:
# Read in CSDA data
csda_df = pd.read_csv(os.path.join(os.pardir, "shared_data", "csda_salesforce_export.csv"))

# Rename columns
csda_df.columns = [
    "_".join(column.lower().split())
    for column in csda_df.columns
]

# Add prefix for clarity after join
csda_df = csda_df.add_prefix("salesforce|")

In [21]:
# Left join CSDA table with main dataframe (df)
df = df.merge(
    csda_df,
    how="left",
    left_on="account|_id.$oid",
    right_on="salesforce|chargebee_id"
)

In [22]:
# Get Hubspot data
hubspot_df = get_hubspot_data()

Fetching Hubspot data...
Hubspot contacts...
Hubspot companies...


In [23]:
# Merge users with HubSpot data
df = df.merge(
    hubspot_df,
    how="left",
    left_on="user|email",
    right_on="hubspot|email"
)

In [24]:
# Add CSDA column
df["salesforce|team_member_name"] = df["salesforce|team_member_name"].fillna("-")
df["salesforce|is_csda"] = df["salesforce|team_member_name"].map(
    lambda x: True if x != "-" else False
)

In [25]:
# Rename columns intended to be kept
column_name_remapping = {
    "account|_id.$oid": "account.id",
    "account|name": "account.name",
    "account|dateCreated.$date": "account.date_created",
    "chargebee|status": "chargebee.status",
    "user|_id.$oid": "user.id",
    "user|language": "user.language",
    "user|dateCreated.$date": "user.date_created",
    "user|email": "user.email",
    "user|auth0Cache.usermetadata.lastName": "user.last_name",
    "user|auth0Cache.usermetadata.firstName": "user.first_name",
    "account|users_count": "account.users_count",
    "hubspot|jobtitle": "hubspot.job_title",
    "hubspot|industry": "hubspot.industry",
    "hubspot|numberofemployees": "hubspot.number_of_employees",
    "salesforce|is_csda": "salesforce.is_csda",
    "hubspot|country": "hubspot.country"
}

# Keep only the keys of mapping columns
df = df[column_name_remapping.keys()]

# Change names of columns based on mapping
df = df.rename(columns=column_name_remapping)

In [26]:
# Change language code to categorical
language_code_mapping = {
    0: "English",
    1: "German",
    2: "Chinese",
    3: "Bulgarian",
    4: "Spanish",
    5: "French",
    6: "Portuguese"
}

df = df.replace({"user.language": language_code_mapping})

In [27]:
df.head()

,account.id,account.name,account.date_created,chargebee.status,user.id,user.language,user.date_created,user.email,user.last_name,user.first_name,account.users_count,hubspot.job_title,hubspot.industry,hubspot.number_of_employees,salesforce.is_csda,hubspot.country
0,5b14ee04854ff50007e2a077,Azavista,2018-06-04 07:45:08.977,active,5b193d8cd340c50007d7c727,English,2018-06-07 14:13:32.294000,jp@azavista.com,van der Kuijl,JP,2,Commercial Director,Electronics,45,False,Netherlands
1,5b14ee04854ff50007e2a077,Azavista,2018-06-04 07:45:08.977,active,5b14ee05854ff50007e2a078,English,2018-06-04 07:45:09,gabriel.thomaidis@azavista.com,Thomaidis,Gabriel,2,People Operations Specialist,Electronics,45,False,Netherlands
2,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5b1e3680d8c5b500075ecfd4,English,2018-06-11 08:44:48.498000,tr@wetransform.to,Reitz,Thorsten,4,"CEO, Founder",Electronics,10,False,Switzerland
3,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5b1fba8dd8c5b500075f2db2,English,2018-06-12 12:20:29.635000,st@wetransform.to,Templer,Simon,4,Co-founder,Electronics,10,False,Switzerland
4,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5d124056d4643c00018d63a8,English,2019-06-25 15:40:06.882000,ch@wetransform.to,Hönn,Christopher,4,None,Electronics,10,False,Switzerland


In [28]:
# Fetch user_roles data
user_roles_df = get_user_roles_table(blob_service_client, today_str)
user_roles_df.head()

,user_roles._id.$oid,user_roles.userId.$oid,user_roles.roleId.$oid,user_roles.accountId.$oid
0,574e9259ed915d0006b985e5,574e9259ed915d0006b985e4,573d93d9ed915d00052efb6b,574e9259ed915d0006b985e3
1,573e222fed915d0005cc2d07,573e222fed915d0005cc2d06,573d93d9ed915d00052efb6b,573e222fed915d0005cc2d05
2,573d93d9ed915d00052efb6c,573d93d9ed915d00052efb6a,573d93d9ed915d00052efb6b,573d93d9ed915d00052efb69
3,5746e3c8ed915d0005cc319c,5746e3c8ed915d0005cc319b,573d93d9ed915d00052efb6b,5746e3c8ed915d0005cc319a
4,573f32bded915d0005cc2e5d,573f32bded915d0005cc2e5c,573d93d9ed915d00052efb6b,573f32bded915d0005cc2e5b


In [29]:
# Fetch roles data
roles_df = get_roles_table(blob_service_client, today_str)
roles_df.head()

,roles._id.$oid,roles.name,roles.accountId.$oid
0,573dbb61ed915d0005cc2c4d,user,NaN
1,5a9f9cbee5274a0007acfcf9,Company B,57fb5f7bed915d0006582898
2,5b86ad85df457100079c04e4,Admin II,57fb5f7bed915d0006582898
3,5b8d2accf9159100080416a2,Air (bo's experiment),573dbb12ed915d0005cc2c46
4,58f632d3ed915d0005e9ef6c,test role,58822288ed915d0005afa6ee


In [30]:
# Join user_roles with roles
user_roles_df = user_roles_df.merge(
    roles_df,
    how="left",
    left_on="user_roles.roleId.$oid",
    right_on="roles._id.$oid"
)

user_roles_df.head()

,user_roles._id.$oid,user_roles.userId.$oid,user_roles.roleId.$oid,user_roles.accountId.$oid,roles._id.$oid,roles.name,roles.accountId.$oid
0,574e9259ed915d0006b985e5,574e9259ed915d0006b985e4,573d93d9ed915d00052efb6b,574e9259ed915d0006b985e3,573d93d9ed915d00052efb6b,admin,NaN
1,574e9259ed915d0006b985e5,574e9259ed915d0006b985e4,573d93d9ed915d00052efb6b,574e9259ed915d0006b985e3,573d93d9ed915d00052efb6b,admin,NaN
2,573e222fed915d0005cc2d07,573e222fed915d0005cc2d06,573d93d9ed915d00052efb6b,573e222fed915d0005cc2d05,573d93d9ed915d00052efb6b,admin,NaN
3,573e222fed915d0005cc2d07,573e222fed915d0005cc2d06,573d93d9ed915d00052efb6b,573e222fed915d0005cc2d05,573d93d9ed915d00052efb6b,admin,NaN
4,573d93d9ed915d00052efb6c,573d93d9ed915d00052efb6a,573d93d9ed915d00052efb6b,573d93d9ed915d00052efb69,573d93d9ed915d00052efb6b,admin,NaN


In [31]:
unique_user_roles_df = get_roles_by_user(user_roles_df)
unique_user_roles_df.head()

,user_roles.userId.$oid,roles
0,573d9359ed915d00052efb10,SysAdmin
1,573d93d9ed915d00052efb6a,admin
2,573db6aeed915d0005cc2bc5,admin
3,573dbb12ed915d0005cc2c47,"admin, user, user+, Ivan Osmak's team_7ba678c2..."
4,573dbb61ed915d0005cc2c4c,"admin, user, user+, Engineering_a8a9b5181dbd8f..."


In [32]:
# Join df with user_roles_df
df = df.merge(
    unique_user_roles_df,
    how="left",
    left_on="user.id",
    right_on="user_roles.userId.$oid"
)

df.head()

,account.id,account.name,account.date_created,chargebee.status,user.id,user.language,user.date_created,user.email,user.last_name,user.first_name,account.users_count,hubspot.job_title,hubspot.industry,hubspot.number_of_employees,salesforce.is_csda,hubspot.country,user_roles.userId.$oid,roles
0,5b14ee04854ff50007e2a077,Azavista,2018-06-04 07:45:08.977,active,5b193d8cd340c50007d7c727,English,2018-06-07 14:13:32.294000,jp@azavista.com,van der Kuijl,JP,2,Commercial Director,Electronics,45,False,Netherlands,5b193d8cd340c50007d7c727,"admin, Management_468cde61fa05e7e0_MembersRole..."
1,5b14ee04854ff50007e2a077,Azavista,2018-06-04 07:45:08.977,active,5b14ee05854ff50007e2a078,English,2018-06-04 07:45:09,gabriel.thomaidis@azavista.com,Thomaidis,Gabriel,2,People Operations Specialist,Electronics,45,False,Netherlands,5b14ee05854ff50007e2a078,"admin, People Operations_8eaf717bc3c07560_Memb..."
2,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5b1e3680d8c5b500075ecfd4,English,2018-06-11 08:44:48.498000,tr@wetransform.to,Reitz,Thorsten,4,"CEO, Founder",Electronics,10,False,Switzerland,5b1e3680d8c5b500075ecfd4,"admin, Revenue_MembersRole, Service_MembersRol..."
3,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5b1fba8dd8c5b500075f2db2,English,2018-06-12 12:20:29.635000,st@wetransform.to,Templer,Simon,4,Co-founder,Electronics,10,False,Switzerland,5b1fba8dd8c5b500075f2db2,"user, Product_MembersRole, Product_0d8de34ddea..."
4,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5d124056d4643c00018d63a8,English,2019-06-25 15:40:06.882000,ch@wetransform.to,Hönn,Christopher,4,None,Electronics,10,False,Switzerland,5d124056d4643c00018d63a8,user


In [33]:
df["roles"] = df["roles"].fillna("")
# df["user.is_okr_champion"] = df["roles"].map(lambda role: True if "champion" in (_role := role.lower()) or ("okr" in _role and "champion" in _role) else False)

In [34]:
# Drop unnecessary columns
"""
Should I drop all id's?
account.id
user.id

user_roles._id.$oid
user_roles.userId.$oid
user_roles.roleId.$oid
user_roles.accountId.$oid
roles._id.$oid
roles.accountId.$oid

"""

df = df[
    df.columns[
        ~df.columns.isin(
            [
                # "account.id",
                # "user.id",
                "user_roles._id.$oid",
                "user_roles.userId.$oid",
                "user_roles.roleId.$oid",
                "user_roles.accountId.$oid",
                "roles._id.$oid",
                "roles.accountId.$oid"
            ]
        )
    ]
]

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35262 entries, 0 to 35261
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   account.id                   35262 non-null  object        
 1   account.name                 35262 non-null  object        
 2   account.date_created         35262 non-null  datetime64[ns]
 3   chargebee.status             35262 non-null  object        
 4   user.id                      35262 non-null  object        
 5   user.language                35262 non-null  object        
 6   user.date_created            35262 non-null  object        
 7   user.email                   35262 non-null  object        
 8   user.last_name               35249 non-null  object        
 9   user.first_name              35250 non-null  object        
 10  account.users_count          35262 non-null  int64         
 11  hubspot.job_title            1560 non-nul

In [36]:
df["user.date_created"] = df["user.date_created"].astype("datetime64[ns]")
df["account.date_created"] = df["account.date_created"].astype("datetime64[ns]")

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35262 entries, 0 to 35261
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   account.id                   35262 non-null  object        
 1   account.name                 35262 non-null  object        
 2   account.date_created         35262 non-null  datetime64[ns]
 3   chargebee.status             35262 non-null  object        
 4   user.id                      35262 non-null  object        
 5   user.language                35262 non-null  object        
 6   user.date_created            35262 non-null  datetime64[ns]
 7   user.email                   35262 non-null  object        
 8   user.last_name               35249 non-null  object        
 9   user.first_name              35250 non-null  object        
 10  account.users_count          35262 non-null  int64         
 11  hubspot.job_title            1560 non-nul

In [38]:
df.head()

,account.id,account.name,account.date_created,chargebee.status,user.id,user.language,user.date_created,user.email,user.last_name,user.first_name,account.users_count,hubspot.job_title,hubspot.industry,hubspot.number_of_employees,salesforce.is_csda,hubspot.country,roles
0,5b14ee04854ff50007e2a077,Azavista,2018-06-04 07:45:08.977,active,5b193d8cd340c50007d7c727,English,2018-06-07 14:13:32.294,jp@azavista.com,van der Kuijl,JP,2,Commercial Director,Electronics,45,False,Netherlands,"admin, Management_468cde61fa05e7e0_MembersRole..."
1,5b14ee04854ff50007e2a077,Azavista,2018-06-04 07:45:08.977,active,5b14ee05854ff50007e2a078,English,2018-06-04 07:45:09.000,gabriel.thomaidis@azavista.com,Thomaidis,Gabriel,2,People Operations Specialist,Electronics,45,False,Netherlands,"admin, People Operations_8eaf717bc3c07560_Memb..."
2,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5b1e3680d8c5b500075ecfd4,English,2018-06-11 08:44:48.498,tr@wetransform.to,Reitz,Thorsten,4,"CEO, Founder",Electronics,10,False,Switzerland,"admin, Revenue_MembersRole, Service_MembersRol..."
3,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5b1fba8dd8c5b500075f2db2,English,2018-06-12 12:20:29.635,st@wetransform.to,Templer,Simon,4,Co-founder,Electronics,10,False,Switzerland,"user, Product_MembersRole, Product_0d8de34ddea..."
4,5b1e3680d8c5b500075ecfd3,wetransform GmbH,2018-06-11 08:44:48.473,active,5d124056d4643c00018d63a8,English,2019-06-25 15:40:06.882,ch@wetransform.to,Hönn,Christopher,4,None,Electronics,10,False,Switzerland,user


In [39]:
# Format datetime fields as string for output file
df["account.date_created"] = df["account.date_created"].dt.strftime("%Y/%m/%d")
df["user.date_created"] = df["user.date_created"].dt.strftime("%Y/%m/%d")

In [40]:
df.shape

(35262, 17)

In [41]:
# Save data to csv file
df.to_csv(f"all_users_180+days_data_{datetime_string(today, '-')} as of {datetime_string(datetime.datetime.now(), '-')}.csv", index=False)